<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 3: Feature Engineering

## Principal Component Analysis (PCA) in Feature Englineering

### Lesson Objectives

After completing this lesson, you should be able to: 

- Understand what Principal Component Analysis (PCA) is
-	Understand PCA's role in feature engineering 



## PCA: Definition 

-	PCA is a dimension reduction technique. it is unsupervised machine learning, and it has many uses; on this video we only care about its use for feature engineering


## PCA: How It Works

-	The first Principal Component (PC) is defined as the linear combination of the predictors that captures the most variability of all possible linear combinations.
-	Then, subsequent PCs are derived such that these linear combinations capture the most remaining variability while also being uncorrelated with all previous PCs.


## Feature Engineering 

-	"Feature Engineering" is a practice where predictors are created and refined to maximize model performance
-	It can take quite some time to identify and prepare relevant features


## Feature Engineering with PCA

-	Basic idea: generate a smaller set of variables that capture most of the information in the original variables
-	The new predictors are functions of the original predictors; all the original predictors are still needed to create the surrogate variables Dataset: Predict US Crimes
-	We want to predict the proportion of violent crimes per 100k population on different locations in the US
-	More than 100 predictors. Examples:
  -	`householdsize`: mean people per household
  - `PctLess9thGrade`: percentage of offenders who have not yet entered high school
  - `pctWWage`: percentage of households with wage or salary income in 1989
-	For a description of these variables, see the UCI repository (communities and crimes) Dataset: Predict US Crimes 
-	Let's assume that we don't want to operate with those >100 predictors. Why?
-	Some will be collinear (ie highly correlated)
- It's hard to see relationships in a high-dimensional space 
-	How do we use PCA to get down to 10 dimensions?

In [ ]:
import sys.process._
"wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0105EN/data/UScrime2-colsLotsOfNAremoved.csv " !

In [ ]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

import org.apache.spark.sql.functions._
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.{VectorAssembler, PCA}

In [ ]:
val crimes = spark.read.
    format("com.databricks.spark.csv").
    option("delimiter", ",").
    option("header", "true").
    option("inferSchema", "true").
    load("UScrime2-colsLotsOfNAremoved.csv")

In [ ]:
val assembler = new VectorAssembler().setInputCols(crimes.columns.filterNot(name => List("community", "otherpercap").contains(name.toLowerCase))).setOutputCol("features")
//this is different from video 
val featuresDF = assembler.transform(crimes).select("features")

val pca = new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(10)
  .fit(featuresDF)

val result = pca.transform(featuresDF).select("pcaFeatures")
result.show(false)

-	Principal components are stored in a local dense matrix.
-	The matrix pc is now 10 dimensions, but it represents the variability 'almost as well' as the previous 100 dimensions


## Pros I

-	Interpretability (!)
-	PCA creates components that are uncorrelated, and Some predictive models prefer little to no collinearity (example linear regression)
-	Helps avoiding the 'curse of dimensionality': Classifiers tend to overfit the training data in high dimensional spaces, so reducing the number of dimensions may help


## Pros II 

- Performance. On further modeling, the computational effort often depends on the number of variables. PCA gives you far fewer variables; this may make any further processing more performant
-	For classification problems PCA can show potential separation of classes (if there is a separation).


## Cons

-	The computational effort often depends greatly on the number of variables and the number of data records. 
-	PCA seeks linear combinations of predictors that maximize variability, it will naturally first be drawn to summarizing predictors that retain most of the variation in the data.


## How Many Principal Components to Use?

-	No simple answer to this question
-	But there are heuristics:
- find the elbow on the graph for dimensions by variance explained 
-	Set up a 'variance explained threshold' (for example, take as many Principal components as needed to explain 95% of the variance



## Tip for Best Practice

-	Always center and scale the predictors prior to performing PCA 9see previous course). Otherwise the predictors that have more validation will soak the top principal components


## Lesson Summary

Having completed this lesson, you should be able to: 

-	Apply PCA in Spark
-	Use PCA to fix datasets with correlated predictors that could otherwise trip your models!

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.